# 02: Ingeniería de Características para Ayatsil (v2)

* **Autor:** Emmanuel Eduardo
* **Fecha de Versión:** 2025-07-20
* **Versión:** 2.0 (Avanzada)
* **Proyecto:** Pronóstico de Producción - CNH

---

## Objetivo
Este notebook toma la serie de tiempo del campo **Ayatsil**, crea un conjunto de **características avanzadas** (rezagos a corto, mediano y largo plazo, estadísticas móviles complejas) y guarda el resultado en la capa GOLD.

## Entradas
* `gold/top_10_campos_timeseries.parquet`

## Salidas
* `gold/ayatsil_features_advanced_v2.parquet`

In [2]:
import pandas as pd
import numpy as np
import os

In [4]:
# --- 1. Configuración del Entorno ---
ruta_base = os.path.abspath(os.path.join(os.getcwd(), "..",".."))
ruta_gold = os.path.join(ruta_base, "gold")


In [6]:
# --- 2. Carga de Datos ---
df_top_10 = pd.read_parquet(os.path.join(ruta_gold, "top_10_campos_timeseries.parquet"))
print("Datos iniciales cargados.")

Datos iniciales cargados.


In [8]:
# --- 3. Preparación Base del Campo Ayatsil ---
campo_elegido = "AYATSIL"
df_campo = df_top_10[df_top_10['campo'] == campo_elegido].copy()
df_campo['fecha'] = pd.to_datetime(df_campo['fecha'])
df_campo.set_index('fecha', inplace=True)

In [10]:
# --- 4. Ingeniería de Características (Simple y Avanzada) ---

# Características simples (v1)
df_campo['produccion_mes_anterior'] = df_campo['petroleo_mbd'].shift(1)
df_campo['produccion_promedio_3m'] = df_campo['petroleo_mbd'].rolling(window=3).mean()
df_campo['mes'] = df_campo.index.month
df_campo['trimestre'] = df_campo.index.quarter

# --- NUEVO: Características Avanzadas (v2) ---
# Rezagos (Lags) a mediano y largo plazo
df_campo['produccion_lag_6'] = df_campo['petroleo_mbd'].shift(6)
df_campo['produccion_lag_12'] = df_campo['petroleo_mbd'].shift(12)

In [12]:
# Estadísticas Móviles (Rolling Stats) más complejas
df_campo['produccion_roll_mean_6'] = df_campo['petroleo_mbd'].rolling(window=6).mean()
df_campo['produccion_roll_std_6'] = df_campo['petroleo_mbd'].rolling(window=6).std()

In [14]:
# Componentes de fecha adicionales
df_campo['año'] = df_campo.index.year

print(f"Ingeniería de características avanzada completada para: {campo_elegido}.")

Ingeniería de características avanzada completada para: AYATSIL.


In [16]:
# --- 5. Limpieza y Guardado del Dataset v2 ---
# Eliminar filas con valores nulos (ahora se eliminarán más por el lag de 12 meses)
df_modelo = df_campo.dropna()

In [20]:
# Definir el nombre del archivo de salida para la v2
nombre_archivo_salida = "ayatsil_features_advanced_v2.parquet"
ruta_completa_salida = os.path.join(ruta_gold, nombre_archivo_salida)

In [24]:
# Guardar el DataFrame final en la capa GOLD
df_modelo.to_parquet(ruta_completa_salida)

In [26]:
print(f"\nProceso finalizado.")
print(f"DataFrame con características avanzadas para {campo_elegido} guardado en:")
print(ruta_completa_salida)


Proceso finalizado.
DataFrame con características avanzadas para AYATSIL guardado en:
C:\Users\elezr\Datalquemy\Proyectos\CNH\gold\ayatsil_features_advanced_v2.parquet
